<a href="https://colab.research.google.com/github/mgultekin/NLPinTensorflow/blob/main/TextGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Author: MustafaGultekin
#Purpose Practicing text generation with lstm through Project Gutenberg
#source: https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/

**Develop a Small LSTM Recurrent Neural Network**

In [1]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [2]:
#load ascii text and convert it to lowercase
filename = "wonderland.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()

In [5]:
#we need to convert chars to integers, so were going to create a map of each charachter
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i,c in enumerate(chars))
char_to_int

In [7]:
n_chars = len(raw_text)
n_vocab = len(chars)

print('Total characters: ', n_chars)
print('Total vocab: ', n_vocab)

Total characters:  163779
Total vocab:  58


In [9]:
# now we need to prepare data for training
# we will split data into subsequences with a fixed length of 100characters

#Each training pattern of the network is comprised of 100 timestemps of one charachter(X) followed by one character output
#for example chapt e,  hapte  r   (for length 5)


seq_length = 100
dataX = []
dataY = []

for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i: i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)

print('Total patterns: ', len(dataX))

Total patterns:  163679


In [10]:
# now we need to prepare our data into form of lstm which is [samples, timestamps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length,1))
#normalize
X = X / float(n_vocab)
#one hot encode the ouput variable
y = np_utils.to_categorical(dataY)

In [12]:
X[0], y[0]

(array([[0.81034483],
        [0.84482759],
        [0.79310345],
        [0.70689655],
        [0.62068966],
        [0.5862069 ],
        [0.87931034],
        [0.01724138],
        [0.65517241],
        [0.89655172],
        [0.87931034],
        [0.62068966],
        [0.77586207],
        [0.56896552],
        [0.62068966],
        [0.84482759],
        [0.65517241],
        [0.12068966],
        [0.86206897],
        [0.01724138],
        [0.55172414],
        [0.74137931],
        [0.68965517],
        [0.5862069 ],
        [0.62068966],
        [0.12068966],
        [0.86206897],
        [0.01724138],
        [0.55172414],
        [0.60344828],
        [0.9137931 ],
        [0.62068966],
        [0.77586207],
        [0.87931034],
        [0.89655172],
        [0.84482759],
        [0.62068966],
        [0.86206897],
        [0.01724138],
        [0.68965517],
        [0.77586207],
        [0.01724138],
        [0.93103448],
        [0.79310345],
        [0.77586207],
        [0

In [13]:
#we can define lstm network now.
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

1


In [15]:

# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [16]:
model.fit(X,y, epochs = 20, batch_size= 128, callbacks=callbacks_list)

Epoch 1/20
1279/1279 [==============================] - 18s 14ms/step - loss: 1.7813

Epoch 00001: loss improved from inf to 1.78125, saving model to weights-improvement-01-1.7813.hdf5
Epoch 2/20
1279/1279 [==============================] - 18s 14ms/step - loss: 1.7760

Epoch 00002: loss improved from 1.78125 to 1.77597, saving model to weights-improvement-02-1.7760.hdf5
Epoch 3/20
1279/1279 [==============================] - 18s 14ms/step - loss: 1.7762

Epoch 00003: loss did not improve from 1.77597
Epoch 4/20
1279/1279 [==============================] - 18s 14ms/step - loss: 1.7714

Epoch 00004: loss improved from 1.77597 to 1.77139, saving model to weights-improvement-04-1.7714.hdf5
Epoch 5/20
1279/1279 [==============================] - 19s 15ms/step - loss: 1.7687

Epoch 00005: loss improved from 1.77139 to 1.76871, saving model to weights-improvement-05-1.7687.hdf5
Epoch 6/20
1279/1279 [==============================] - 19s 15ms/step - loss: 1.7631

Epoch 00006: loss improved fr

**Generating Text with LSTM Network**

In [17]:
#load model weights
filename = 'weights-improvement-20-1.7138.hdf5'
model.load_weights(filename)

In [18]:
#reverse mapping
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [30]:
#pick a random seed
start = numpy.random.randint(0, len(dataX) - 1)
pattern = dataX[start]
print('seed:')
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

import sys
#generate charachters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose = 0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

seed:
" ime, and was a little
startled when she heard her voice close to her ear. 'you're thinking
about som "
e tome tome tome tome tome tome tome tome tome tome thme to the tares if the coor tiate whsh the toede of the coor, and the door tas a little soaeenly the coor of the court, and the door tas a little soaeenly the coor of the court, and the door tas a little soaeenly and the taid to the court, and the door tas a little soaeenly and the taid to the court, and the door tas a little soaeenly and the taid to the court, and the door tas a little soaeenly and the taid to the court, and the door tas a little soaeenly and the taid to the court, and the door tas a little soaeenly and the taid to the court, and the door tas a little soaeenly and the taid to the court, and the door tas a little soaeenly and the taid to the court, and the door tas a little soaeenly and the taid to the court, and the door tas a little soaeenly and the taid to the court, and the door tas a little soaeenly an

In [ ]:
#just an example for larger LSTM RNN
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')